# Training deep learning model using PyTorch Lightning

This example demonstrates Pytorch Lightning based model training.

It's main focus is to show how the components described in `Implementing and training a model in PyTorch` section can be put together.

Therefore model architecture and data processing is simplified that the trained model does not work.

## Data loading logics

For this example, we will google scholar data that we crawled in Chapter 2

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("sample_google_scholar.csv")

In [3]:
data = data.dropna()
data.head()

,author_name,email,affiliation,coauthors_names,research_interest
0,Lawrence Holder,wsu.edu,Washington State University,Diane J Cook##William Eberle,artificial_intelligence##machine_learning##dat...
3,Diane J Cook,eecs.wsu.edu,Washington State University,Lawrence Holder##Parisa Rashidi##Sajal K. Das#...,artificial_intelligence##machine_learning##sma...
4,Sumi Helal IEEE Fellow AAAS Fellow IET Fellow ...,cise.ufl.edu,University of Florida,Raja Bose##Darrell Woelk##Diane J Cook##Yousse...,digital_health##smart_homes##internet_of_thing...
5,Hani Hagras,essex.ac.uk,University of Essex,Christian Wagner,explainable_artificial_intelligence##ambient_i...
6,Anupam Joshi,umbc.edu,UMBC,Tim Finin##Yelena Yesha##Lalana Kagal##Dipanja...,data_management##mobile_computing##security##s...


In [4]:
# for features, we will convert first 10 characters of affiliation into a vector of float 
# by dividing each character by maximum axcii number (256)

def convert_first_ten_characters_into_tensor(data):
    first_ten_characters = data[:10]
    converted = [ord(char)/256 for char in first_ten_characters]
    while len(converted) < 10:
        converted.append(0.0)
    return np.array(converted)

converted_affiliation = data['affiliation'].map(convert_first_ten_characters_into_tensor)
affiliation = np.vstack(converted_affiliation.values)
print(affiliation[:5])

[[0.33984375 0.37890625 0.44921875 0.40625    0.41015625 0.4296875
  0.40234375 0.453125   0.43359375 0.4296875 ]
 [0.33984375 0.37890625 0.44921875 0.40625    0.41015625 0.4296875
  0.40234375 0.453125   0.43359375 0.4296875 ]
 [0.33203125 0.4296875  0.41015625 0.4609375  0.39453125 0.4453125
  0.44921875 0.41015625 0.453125   0.47265625]
 [0.33203125 0.4296875  0.41015625 0.4609375  0.39453125 0.4453125
  0.44921875 0.41015625 0.453125   0.47265625]
 [0.33203125 0.30078125 0.2578125  0.26171875 0.         0.
  0.         0.         0.         0.        ]]


In [5]:
# for labels, it will be boolean value; True if email consists of '.edu' and False otherwise

converted_email = data['email'].str.contains('.edu')
labels = converted_email.values
labels[:5]

array([ True,  True,  True, False,  True])

In [6]:
from pytorch_lightning import LightningDataModule
from typing import Optional
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn


class SampleDataset(Dataset):
    def __init__(self):
        self.affiliation = torch.Tensor(affiliation)
        self.labels = torch.Tensor(labels)

    def __len__(self):
        """return number of samples"""
        return len(self.labels)

    def __getitem__(self, idx):
        """loads and returns a sample from the dataset at the given index"""
        return self.affiliation[idx], int(self.labels[idx])


class SampleDataModule(LightningDataModule):
    def __init__(self):
        super().__init__()
        
        self.batch_size = 10
        self.collate_fn = lambda x: x

    def prepare_data(self):
        """download and preprocess the data; triggered only on single GPU"""
        pass

    def setup(self, stage: Optional[str] = None):
        """define necessary components for data loading on each GPU"""
        pass
        
    def train_dataloader(self):
        """define train data loader"""
        return DataLoader(
            SampleDataset(),
            batch_size=self.batch_size,
            shuffle=True)
    
    def val_dataloader(self):
        """define validation data loader"""
        return DataLoader(
            SampleDataset(),
            batch_size=self.batch_size,
            shuffle=True)

    def test_dataloader(self):
        """define test data loader"""
        return DataLoader(
            SampleDataset(),
            batch_size=self.batch_size,
            shuffle=False)

## Model definition & Model training 

For this example, we simply have three layers of fully connected layers

In [7]:
from pytorch_lightning.core.lightning import LightningModule
from torch.optim.adam import Adam
from torchmetrics.functional import accuracy


class SampleModel(LightningModule):
    
    def __init__(self):
        super().__init__()
        
        # define Network
        self.layers = nn.Sequential(
            nn.Linear(10, 5),
            nn.Linear(5, 3),
            nn.Linear(3, 1),
            nn.Sigmoid()
        )
        
        # define loss function (BCELoss)
        self.criterion = torch.nn.BCELoss()
        
    def forward(self, x):
        """Feed input tensor to the networks and compute output"""
        output_tensor = self.layers(x)
        return output_tensor
    
    def configure_optimizers(self):
        """Define optimizer to use"""
        return torch.optim.Adam(model.parameters(), lr=0.1)
    
    def training_step(self, batch, batch_idx):
        """Define single training iteration"""
        x, y = batch
        # reformat targets for BCELoss
        targets = y.unsqueeze(dim=1).to(torch.float32)
        outputs = self(x)
        loss = self.criterion(outputs, targets)
        return loss

    def validation_step(self, batch, batch_idx):
        """Define single validation iteration"""
        loss, acc = self._shared_eval_step(batch, batch_idx)
        metrics = {"val_acc": acc, "val_loss": loss}
        self.log_dict(metrics)
        return metrics

    def test_step(self, batch, batch_idx):
        """Define single test iteration"""
        loss, acc = self._shared_eval_step(batch, batch_idx)
        metrics = {"test_acc": acc, "test_loss": loss}
        self.log_dict(metrics)
        return metrics

    def _shared_eval_step(self, batch, batch_idx):
        x, y = batch
        # reformat targets for BCELoss
        targets = y.unsqueeze(dim=1).to(torch.float32)
        outputs = self(x)
        loss = self.criterion(outputs, targets)
        acc = accuracy(outputs.round(), targets.int())
        return loss, acc

    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        """Compute prediction for the given batch of data"""
        x, y = batch
        y_hat = self(x)
        return y_hat


In [8]:
from pytorch_lightning import Trainer

# setup model training
num_epochs = 10
data_module = SampleDataModule()
model = SampleModel()
trainer = Trainer(max_epochs=num_epochs)

# training a model
trainer.fit(model, data_module)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name      | Type       | Params
-----------------------------------------
0 | layers    | Sequential | 77    
1 | criterion | BCELoss    | 0     
-----------------------------------------
77        Trainable params
0         Non-trainable params
77        Total params
0.000     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:655: UserWarning: Your `val_dataloader` has `shuffle=True`, it is strongly recommended that you turn this off for val/test/predict dataloaders.
  rank_zero_warn(
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/opt/conda/lib/python

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [9]:
# evaluate model on test set
result = trainer.test(model, data_module)

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.6333333253860474, 'test_loss': 0.6645503640174866}
--------------------------------------------------------------------------------


In [14]:
# prediction using trained model
model.eval()
preds = model(torch.Tensor(affiliation[:5])).round()
for label, pred in zip(labels[:5], preds):
    print(f"label: {label}, prediction: {bool(pred[0])}")

label: True, prediction: True
label: True, prediction: True
label: True, prediction: True
label: False, prediction: True
label: True, prediction: True
